# Deep Reinforcement learning for the "Dollar Game" environment

In [1]:
# import the Graph which is the base for our environment
import networkx as nx
from Graph import DGGraph, generate_game
from Graph import load_game

# import the classes for ML
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Flatten
# from tensorflow.keras.optimizers import Adam

# utils
import numpy as np
from matplotlib import pyplot as plt
from random import choice
from copy import deepcopy

## Problem setup

### Environment
A graph (DGGraph object) representing a given game. The graph has nodes (with integer values assigned to them) and edges.\
The state is a dictionary {node: node[value] for each node in graph}. 

### Objective
Win the game by making debt (sum of all negative values) non-negative in as few moves as possible (avoid inverse actions; namely, both give and take actions on the same node)

### Action space
On each node, two actions can be taken: 'give' and 'take'.\
'Give' action increases value of the node's neighbors by 1 and subtracts len(neighbors) from its own value.
'Take' action does the complete opposite.\
Thus, $A = \left\{(\text{node},\;\text{give_or_take}): \text{node}\in\text{nodes}, \;\text{give_or_take}\in[\text{give}, \;\text{take}]\right\}$

### Reward function
IDK lol. Probably, something to do with debt (and/or the number of negative values)

## Graph visualization

In [2]:
def print_graph(G):
    for n in G.nodes:
        print(n, G.nodes[n])

In [3]:
def scale_down_positions(pos):
    x, y = pos
    return (x - 160)/640 - 0.5, 1 - y/600 - 0.5

In [4]:
def draw_graph(G, transform=True, savefig=''):
    pos = {node: scale_down_positions(node_data['pos']) if transform else node_data['pos'] \
        for node, node_data in G.nodes.items()}
    fig, ax = plt.subplots(figsize=(7, 7))
    for node, node_data in G.nodes.items():
        plt.text(pos[node][0] + 0.02, pos[node][1] + 0.02, str(node_data['val']), fontsize=14)
        # TODO: fix the shift of values' labels
    nx.draw_networkx(G, pos=pos, ax=ax, node_size=250)
    if savefig:
        fig.savefig(f'others/{savefig}')

In [5]:
def random_list_of_values1(n, bank=0):
    # TODO: generate a list of n integers totalling to bank (fix this!)
    while True:
        a = np.random.randint(-4, 5, n)
        if sum(a) == bank and sum(a < 0):
            return a.tolist()

In [6]:
def random_list_of_values2(n, bank=0):
    # this one is 10 times as fast as v1
    a = np.random.randint(-3, 4, n)
    diff = sum(a) - bank
    shifts = (2, 1) if diff > 0 else (-2, -1)
    for _ in range(abs(diff)):
        inds = np.random.choice(np.arange(len(a)), 2)
        a[inds[0]] -= shifts[0]
        a[inds[1]] += shifts[1]
    return a

## Pseudorandom actions
(negative values take, positive give)

In [7]:
def collapse_moves(moves):
    collapsed = dict()
    for node, move in moves:
        if not node in collapsed.keys():
            collapsed[node] = 0
        collapsed[node] += 1 if move == 'take' else -1
    num_moves = sum([abs(num) for num in collapsed.values()])
    return collapsed, num_moves

In [15]:
def solve(G):
    done = False
    moves = []
    while not done:
        n = choice(list(G.nodes))
        if G.nodes[n]['val'] > 0:
            G.give(n)
            moves.append((n, 'give'))
        elif G.nodes[n]['val'] < 0:
            G.take(n)
            moves.append((n, 'take'))
        if G.is_victory():
            done = True
    return collapse_moves(moves), len(moves)
    

In [9]:
def solve2(G):
    done = False
    moves = []

In [10]:
def show_instruction(moves):
    for node, move in moves.items():
        if move < 0:
            print(f'Node {node} gives {abs(move)} times')
        elif move > 0:
            print(f'Node {node} takes {abs(move)} times')

In [16]:
def find_best(G, N=1000):
    s = 0
    best_so_far = None
    min_num_of_moves = np.inf
    for _ in range(N):
        g = deepcopy(G)
        (moves, number_of_moves), tot_len = solve(g)
        s += tot_len
        if number_of_moves < min_num_of_moves:
            best_so_far = moves
            min_num_of_moves = number_of_moves
    average_non_collapsed_len = s / N
    return best_so_far, min_num_of_moves, average_non_collapsed_len

In [25]:
G = load_game('7.json')
# G, name = get_random_game()
# draw_graph(G), name

In [26]:
moves_best, num_best, avg_non_collapsed = find_best(G, N=1000)
print(f'Number of moves: {num_best}\nAverage non collapsed moves sequence length: {avg_non_collapsed}')
show_instruction(moves_best)

Number of moves: 7
Average length of non collapsed moves sequence: 37.215
Node 3 gives 1 times
Node 0 takes 1 times
Node 2 gives 1 times
Node 5 takes 2 times
Node 4 takes 1 times
Node 6 gives 1 times
